----

# TUIA 2024 - COMPUTER VISION

# PRÁCTICA UNIDAD 1

Eugenio M. Lopez

----



In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Preparación del entorno.
Acceso al repositorio de la unidad. Lo clonamos y cd al repo.

In [ ]:
import os

REPO_NAME = "computer_vision"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/euglpz/{REPO_NAME}.git
  os.chdir(REPO_NAME)

In [ ]:
# Me muevo a la carpeta de la unidad 1
os.chdir('unidad_1')

# Ejercicio 1

Elija una de las imágenes color que tomó para la clase y aplique separación de canales y elija un método para transformarla en escala de grises. Muestre por pantalla los resultados obtenidos.

In [ ]:
# Cargar la imagen
image = cv2.imread('img_simple.jpg')

# Convertir la imagen a RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
# Separación de canales rgb (alto, ancho, canal)
r, g, b = image_rgb[:,:,0], image_rgb[:,:,1], image_rgb[:,:,2]

In [ ]:
# Muestra de imágenes por canal
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(r,cmap='Reds')
plt.title('R')
plt.subplot(1, 3, 2)
plt.imshow(g,cmap='Greens')
plt.title('G')
plt.subplot(1, 3, 3)
plt.imshow(b, cmap='Blues')
plt.title('B')

plt.show()

In [ ]:
# Convertir la imagen a escala de grises
image_gr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Imagen en escala de grises
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image_gr, cmap='Greys')

# Ejercicio 2
Con las fotografías pedidas por la cátedra la clase pasada (la foto de objetos con fondo liso, y fotos del mismo producto en un contexto más complejo) usar los métodos de extracción de características (esos anteriores al Deep Learning) para encontrar la ubicación del producto dentro de la imagen.

In [ ]:
# Cargo imagen
img = cv2.imread('img_simple.jpg')

# Escala de grises
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Inicializo objeto SIFT
sift = cv2.SIFT_create()
kp = sift.detect(gray, None) # Keypoints

img_with_sift_kp = cv2.drawKeypoints(gray, kp, img)

# Guardo la img con los kp
cv2.imwrite('sift_keypoints.jpg', img)

In [ ]:
# Usar matplotlib para mostrar las imágenes
plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
plt.imshow(img_with_sift_kp)
plt.title('SIFT Keypoints')

plt.subplot(1, 2, 2)
plt.imshow(gray, cmap='gray')
plt.title('Imagen original ByN')

plt.show()

Keypoint usando flag para dibujar circulos

In [ ]:
img_with_sift_kp_flag = cv2.drawKeypoints(gray, kp, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('sift_keypoints_2.jpg',img)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(img_with_sift_kp_flag)
plt.title('SIFT Keypoints')

plt.subplot(1, 2, 2)
plt.imshow(gray, cmap='gray')
plt.title('Imagen original ByN')

plt.show()

Extracción características de otra imagen

In [ ]:
img = cv2.imread('img_objetos_detras.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
kp = sift.detect(gray,None)

img_2_with_sift_kp_flag = cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('sift_keypoints_3.jpg',img)


In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1,2,1)
plt.imshow(img_2_with_sift_kp_flag)
plt.title('SiFT keypoints')

plt.subplot(1,2,2)
plt.imshow(gray, cmap='gray')
plt.title('Img orig')
plt.show()

Keypoints matching entre ambas imágenes usando FLANN (Fast Library for Approximate Nearest Neighbors)

In [ ]:
from __future__ import print_function
import cv2 as cv
import numpy as np

img1 = cv.imread('img_simple.jpg', cv.IMREAD_GRAYSCALE)
# img2 = cv.imread('img_tapada.jpg', cv.IMREAD_GRAYSCALE)
img2 = cv.imread('img_objetos_detras_3.jpg', cv.IMREAD_GRAYSCALE)

if img1 is None or img2 is None:
  print('Could not open or find the images!')
  exit(0)

# Primer paso: detección de keypoints usando SIFT y calculo de descriptores
minHessian = 400
detector = cv.SIFT_create()
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)

# Segundo paso: matcheo de vectores descriptores usando FLANN
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)

# Filtrar las coincidencias mediante Lowe ratio test
ratio_thresh = 0.7
good_matches = []
for m,n in knn_matches:
  if m.distance < ratio_thresh * n.distance:
    good_matches.append(m)

# Dibujar coincidencias
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Mostrar coincidencias detectadas
plt.figure(figsize=(16, 8))
plt.imshow(img_matches, cmap='Greys')

# Guardo imagen final
cv.imwrite('finish.jpg',img_matches)

cv.waitKey()

# Ejercicio 4
Con los videos de youtube.com de cámara fija pedidos para esta clase, aplicar los algoritmos de detección de movimiento vistos en la teoría

In [ ]:
%%capture
!pip install pytube
!apt update && apt install -y handbrake

In [ ]:
from pytube import YouTube

# URL del video de YouTube
url = 'https://www.youtube.com/watch?v=fUHmDs-a1Z8'

# Crear un objeto YouTube
yt = YouTube(url)

# Seleccionar el stream con resolución de 720p
video = yt.streams.filter(progressive=True, file_extension='mp4', res="720p").first()

# Nombre del archivo de salida
output_filename = 'video.mp4'

# Descargar el video con el nombre especificado
video.download(filename=output_filename)

print("Descarga completada.")

In [ ]:
%%capture
#Esto convierte el video a un formato más liviano de trabajo
!ffmpeg -y -i video.mp4 -vf "scale=600:-1" -an -t 30 video_600.mp4

Prueba del video


In [ ]:
!pip install -q mediapy
import mediapy as media

url = 'video_600.mp4'
video = media.read_video(url)
media.show_video(video)

In [ ]:
import mediapy as media  # Importar la biblioteca mediapy para manejo de medios
import cv2  # Importar OpenCV para el procesamiento de imágenes y videos
import numpy as np


# Función para oscurecer una imagen:
def process_image(new_image, prev_image, **kwargs):
    # Convertir la imagen a float32
    new_image_float = new_image.astype(np.float32)

    # Reducir el brillo de la imagen a la mitad
    new_image_float *= 0.5

    # Convertir la imagen de vuelta a uint8
    new_image_uint8 = np.clip(new_image_float, 0, 255).astype(np.uint8)

    return new_image_uint8

def draw_contours(frame, contours, color=(0, 255, 0), thickness=2):
    # Comprobar si la imagen es en escala de grises (1 canal)
    if len(frame.shape) == 2 or frame.shape[2] == 1:
        # Convertir la imagen de escala de grises a color (3 canales)
        result_image = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    else:
        # Si ya es una imagen de color, simplemente hacer una copia
        result_image = frame.copy()

    # Dibujar cada contorno en la imagen
    for contour in contours:
        # Obtener el rectángulo delimitador para cada contorno
        x, y, w, h = cv2.boundingRect(contour)
        # Dibujar el rectángulo
        cv2.rectangle(result_image, (x, y), (x + w, y + h), color, thickness)

    return result_image

# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas
            prev_image = None  # Inicializar la imagen previa

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # Actualizar la imagen previa
                prev_image = new_image.copy()

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

# Nombres de los archivos de video de entrada y salida
filename_in = 'video_600.mp4'
filename_out = 'video_dark.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_image, 10)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

In [ ]:
# Función actualizada para realizar diferencia de fotogramas con normalización:
def process_frame_difference(new_image, prev_image, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # Convertir la imagen umbralizada a color para mantener la consistencia con el video original
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

    return thresh_color

# Nombres de los archivos de video de entrada y salida
filename_in = 'video_600.mp4'
filename_out = 'video_frame_difference.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_frame_difference, 10)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

In [ ]:
# Función actualizada para detectar movimientos y dibujar cuadros delimitadores:
def process_frame_difference_full(new_image, prev_image, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # Dilatar la imagen umbralizada para mejorar la detección de contornos
    kernel = np.ones((5,5),np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations = 1)

    # Convertir la imagen dilatada a formato adecuado para findContours
    dilated = dilated.astype(np.uint8)

    # Encontrar contornos en la imagen dilatada
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Dibujar cuadros delimitadores alrededor de los contornos
    if kwargs.get('draw_mode', 0) == 0:
      result_image = draw_contours(new_image, contours)
    elif kwargs.get('draw_mode', 0) == 1:
      result_image = draw_contours(thresh, contours)

    return result_image


# Nombres de los archivos de video de entrada y salida
filename_in = 'video_600.mp4'
filename_out = 'video_frame_difference_full.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_frame_difference_full,
                max_time=10, draw_mode=1)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

# Llamar a la función para procesar el video
filename_out = 'video_frame_difference_full_2.mp4'
video_processor(filename_in, filename_out, process_frame_difference_full,
                max_time=10, draw_mode=0)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

# Ejercicio 5
Genere un video en un patio o en un hall de edificio donde en un principio se vea vacío y luego aparezca una persona. Mediante los métodos de motion detection (sin usar deep learning) logre una detección de la persona cuando entra al cuadro suponiendo la utilidad para una cámara de seguridad.
Luego sobre el mismo video aplique los algoritmos de flujo denso y disperso que se mostraron en clase.
Escriba una reflexión sobre los resultados en el formato md dentro del Jupyter Notebook.

En este caso, el video que grabé lo subí a youtube para poder acceder a él más fácilmente ya que no podía subirlo local o a GitHub porque excedía en el tamaño.

In [ ]:
%%capture
!pip install pytube
!apt update && apt install -y handbrake

In [ ]:
from pytube import YouTube

# URL del video de YouTube
url = 'https://www.youtube.com/shorts/9KOunnWc2Tg'

# Crear un objeto YouTube
yt = YouTube(url)

# Seleccionar el stream con resolución de 720p
video = yt.streams.filter(progressive=True, file_extension='mp4', res="720p").first()

# Nombre del archivo de salida
output_filename = 'video_ejer_5.mp4'

# Descargar el video con el nombre especificado
video.download(filename=output_filename)

print("Descarga completada.")

In [ ]:
#Esto convierte el video a un formato más liviano de trabajo
!ffmpeg -y -i video_ejer_5.mp4 -vf "scale=600:-1" -an -t 30 video_ejer_5_600.mp4

Descargo el video en menor resolución/calidad

In [ ]:
# from google.colab import files

# # Descargar el archivo de video
# files.download('/content/video_ejer_5_600.mp4')

Prueba del video

In [ ]:
!pip install -q mediapy
import mediapy as media

In [ ]:
url = 'video_ejer_5_600.mp4'
video = media.read_video(url)
media.show_video(video)

Reutilizo funciones anteriores para realizar el procesamiento del video



In [ ]:
# Función para oscurecer una imagen:
def process_image(new_image, prev_image, **kwargs):
    # Convertir la imagen a float32
    new_image_float = new_image.astype(np.float32)

    # Reducir el brillo de la imagen a la mitad
    new_image_float *= 0.5

    # Convertir la imagen de vuelta a uint8
    new_image_uint8 = np.clip(new_image_float, 0, 255).astype(np.uint8)

    return new_image_uint8

def draw_contours(frame, contours, color=(0, 255, 0), thickness=2):
    # Comprobar si la imagen es en escala de grises (1 canal)
    if len(frame.shape) == 2 or frame.shape[2] == 1:
        # Convertir la imagen de escala de grises a color (3 canales)
        result_image = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    else:
        # Si ya es una imagen de color, simplemente hacer una copia
        result_image = frame.copy()

    # Dibujar cada contorno en la imagen
    for contour in contours:
        # Obtener el rectángulo delimitador para cada contorno
        x, y, w, h = cv2.boundingRect(contour)
        # Dibujar el rectángulo
        cv2.rectangle(result_image, (x, y), (x + w, y + h), color, thickness)

    return result_image

# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas
            prev_image = None  # Inicializar la imagen previa

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # Actualizar la imagen previa
                prev_image = new_image.copy()

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

In [ ]:
# Nombres de los archivos de video de entrada y salida
filename_in = 'video_ejer_5_600.mp4'
filename_out = 'video_ejer_5_dark.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_image, 10)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

## Motion detection

In [ ]:
def process_frame_difference_full(new_image, prev_image, min_area_threshold=13000, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # --- FILTRO BLUR (homogeneizar fondo)
    img_blur = cv2.medianBlur(norm_diff, 5, 2)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(img_blur, 60, 255, cv2.THRESH_BINARY)

    # # # Apertura para eliminar pequeñas porciones de píxeles
    # opening_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    # open = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, opening_kernel)

    # Dilatar la imagen umbralizada para mejorar la detección de contornos
    # kernel = np.ones((5,5),np.uint8)
    opening_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (23,15))
    dilated = cv2.dilate(thresh, opening_kernel, iterations = 3)

    # Convertir la imagen dilatada a formato adecuado para findContours
    dilated = dilated.astype(np.uint8)

    # Encontrar contornos en la imagen dilatada
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar los contornos por área mínima
    filtered_contours = []
    for contour in contours:
        if cv2.contourArea(contour) >= min_area_threshold:
            filtered_contours.append(contour)

    # Dibujar cuadros delimitadores alrededor de los contornos
    if kwargs.get('draw_mode', 0) == 0:
      result_image = draw_contours(new_image, filtered_contours)
    elif kwargs.get('draw_mode', 0) == 1:
      result_image = draw_contours(thresh, filtered_contours)

    return result_image

In [ ]:
# Nombres de los archivos de video de entrada y salida
filename_in = 'video_ejer_5_600.mp4'
filename_out = 'video_frame_difference_full.mp4'

# # Llamar a la función para procesar el video
# video_processor(filename_in, filename_out, process_frame_difference_full,
#                 max_time=10, draw_mode=1)

# # Mostrar el video resultante
# media.show_video(media.read_video(filename_out), fps=30)

#Llamar a la función para procesar el video
filename_out = 'video_frame_difference_full_2.mp4'
video_processor(filename_in, filename_out, process_frame_difference_full,
                max_time=10, draw_mode=0)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

## Sparse flow


In [ ]:
import mediapy

def process_sparse_optical_flow(new_image, prev_image):
    # Preparamos las imagenes de trabajo
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    prev_gray_image = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)

    img_blur = cv2.medianBlur(new_gray, 5, 2)

    # Verificar si ya se han detectado las características de Shi-Tomasi
    if not hasattr(process_sparse_optical_flow, "shi_tomasi_done"):
        # Definir parámetros para la detección de esquinas de Shi-Tomasi
        feature_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)
        # Detectar puntos característicos en la imagen
        process_sparse_optical_flow.prev_points = cv2.goodFeaturesToTrack(img_blur, mask=None, **feature_params)
        # Crear una máscara para dibujar el flujo óptico
        process_sparse_optical_flow.mask = np.zeros_like(new_image)
        # Marcar que se ha completado la detección de Shi-Tomasi
        process_sparse_optical_flow.shi_tomasi_done = True

    # Continuar si se ha completado la detección de Shi-Tomasi
    if process_sparse_optical_flow.shi_tomasi_done:
        prev_points = process_sparse_optical_flow.prev_points
        mask = process_sparse_optical_flow.mask

    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calcular el flujo óptico de Lucas-Kanade
    new_points, status, error = cv2.calcOpticalFlowPyrLK(prev_gray_image, img_blur, prev_points, None, **lk_params)
    # Filtrar puntos buenos
    good_old = prev_points[status == 1]
    good_new = new_points[status == 1]
    color = (0, 255, 0)  # Color para el dibujo
    # Dibujar el movimiento (flujo óptico)
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.astype(int).ravel()
        c, d = old.astype(int).ravel()
        mask = cv2.line(mask, (a, b), (c, d), color, 2)
        new_image = cv2.circle(new_image, (a, b), 3, color, -1)

    # Combinar la imagen actual con las líneas de flujo óptico dibujadas
    output = cv2.add(new_image, mask)
    # Actualizar puntos para el siguiente cuadro
    process_sparse_optical_flow.prev_points = good_new.reshape(-1, 1, 2)
    return output

# Nombres de los archivos de video de entrada y salida
filename_in = 'video_ejer_5_600.mp4'
filename_out = 'video_ejer_5_600_sparse_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_sparse_optical_flow,
                max_time=10)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

## Dense flow

In [ ]:
# Función para procesar el flujo óptico denso
def process_dense_optical_flow(new_image, prev_image):
    # Convierte la nueva imagen a escala de grises
    gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    if not hasattr(process_dense_optical_flow, "init_done"):
        process_dense_optical_flow.prev_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        process_dense_optical_flow.mask = np.zeros_like(new_image)
        process_dense_optical_flow.mask[..., 1] = 255
        process_dense_optical_flow.init_done = True

    if process_dense_optical_flow.init_done:
        prev_gray = process_dense_optical_flow.prev_gray
        mask = process_dense_optical_flow.mask

    # Calcula el flujo óptico
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Computa magnitud y ángulo de los vectores 2D
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Establece el tono de la imagen según la dirección del flujo óptico
    mask[..., 0] = angle * 180 / np.pi / 2
    # Establece el valor de la imagen según la magnitud del flujo óptico
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convierte de HSV a RGB
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    # Actualiza la imagen previa a gris
    process_dense_optical_flow.prev_gray = gray.copy()
    return rgb

# Nombres de los archivos de video de entrada y salida
filename_in = 'video_ejer_5_600.mp4'
filename_out = 'video_ejer_5_600_dense_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, process_dense_optical_flow,
                max_time=20)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

Los resultados obtenidos acorde a la **detección de movimiento** fueron bastante satisfactorios. Se pudo detectar y segmentar correctamente el momento en el que la persona aparece en escena, como así también se la detecta cuando se va moviendo por el cuadro.

Se tuvieron una serie de inconvenientes con los que lidiar:

En primer lugar, había un poco de viento cuando se grabó el video, por lo tanto las hojas de las plantas se movian un poco. Esto generaba ruido a la hora de procesar el video y en consecuencia, hacía que se detecte ese movimiento también y no era lo que se quería. Para resolverlo, agregue un filtro BLUR para homogeneizar el fondo y de esta manera se elimino gran parte de ese ruido. Además agregué también el parámetro *min_area_threshold* ya que aún se detectaba algo de ruido y de esta forma, el contorno se dibuja en objetos a partir de ese límite establecido (seteado en un valor bastante alto).

Se probó también aplicando una apertura antes de la dilatación, pero esto hacía desaparecer gran parte de los píxeles de la figura de la persona y luego no se podía llegar a segmentar correctamente.

Después de jugar bastante tiempo con los parámetros tanto del filtro blur, como del umbral y de la dilatación se llegó al resultado que se muestra.

Otro problema que se tiene es que, en la detección de movimiento, se detecta también la sombra de la persona, esto puede ser una contra como también algo bueno, ya que si por algún motivo la persona nunca aparece en escena, la cámara de seguridad podría detectar de todas formas que hay alguien en ese lugar sólo por su sombra.

---

Para el caso del **Flujo disperso** también se aplicó un filtro blur para homogeneizar el fondo y se obtuvo un buen resultado donde se determina el flujo de movimiento de la persona en el video. Se observa que la detección de movimiento se da en partes puntuales de la figura, como en las manos, cabeza y pies.

---

En el caso del **Flujo denso** también se obtuvo un buen resultado al determinar el flujo de movimiento pero en este caso, se tardó bastante más tiempo en finalizar la ejecución del algoritmo, llegando a los 2 min. cuando en el caso anterior se completó en unos 25 seg. También se observa que se detecta el movimiento de la sombra en esta oportunidad.

#Ejercicio 6
Explique cuál es diferencia entre localización de objetos y clasificación de imágenes. Muestre ejemplos de ello.

## Diferencias entre clasificación de imágenes y localización de objetos:

Los modelos de **clasificación de imágenes** son algoritmos diseñados para identificar y etiquetar o clasificar objetos o características dentro de imágenes, es decir, estos modelos toman una imagen como entrada y predicen la clase o categoría a la que pertenece esa imagen en su totalidad, lo hacen sin precisión, o sea que nos limita a saber la cantidad de instancias que hay, donde estan ubicados, etc. La salida en los modelos de clasificación de imágenes suele ser una sola etiqueta que representa la categoría dominante de la imagen de entrada.
Por ejemplo, en un conjunto de imágenes de perros y gatos, la clasificación de imágenes determinaría si la imagen muestra un perro o un gato.


En cambio, los modelos de **localización de objetos** combinan la clasificación de imágenes y la localización para identificar múltiples objetos en imágenes y determinar su posición exacta a través de un bounding box.
La localización de objetos supera a la clasificación de imágenes en importancia práctica, ya que permite ubicar objetos en la imagen, para su posterior análisis, modificación o clasificación. Continuando con el ejemplo de una imagen con perros y gatos, la localización de objetos identificaría la presencia de cada animal y delimitaría un cuadro alrededor de ellos para indicar su ubicación y extensión.






## Ejemplos

### Clasificación de imágenes:

 Para este ejemplo voy a utilizar una foto de mi perro.

In [ ]:
# Muestro la imagen
img = cv2.imread('foto_img_classif.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')
plt.show()

Utilizo el modelo preentrenado de EfficientNet80 para luego usarlo para clasificar mi imagen.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

# Cargar el modelo preentrenado EfficientNetB0
model = EfficientNetB0(weights='imagenet')

def classify_image(img_path):
    # Cargar y preparar la imagen
    img = Image.open(img_path)
    img = img.resize((224, 224))  # Tamaño de entrada para EfficientNetB0

    # Convertir la imagen a un array y procesarla
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Realizar la predicción con el modelo
    predictions = model.predict(img_array)

    # Decodificar y retornar las predicciones
    return decode_predictions(predictions, top=3)[0]

In [ ]:
# Accedo a la imagen contenida en el repo
img_path = 'foto_img_classif.jpg'
predictions = classify_image(img_path)

print('\n')

# Mostrar las predicciones
for i, (imagenet_id, label, score) in enumerate(predictions):
    print(f"{i + 1}: {label} ({score:.2f})")

# img = cv2.imread('foto_img_classif.jpg')

El modelo clasificó correctamente la imagen de mi perro como un Boxer.

### Localización de objetos:

Para este caso voy a utilizar una foto de mi novia andando en bicicleta por la calle.

In [ ]:
%pip install ultralytics
# Librería mediapy para mostrar videos
%pip install -q mediapy
import ultralytics
ultralytics.checks()

In [ ]:
# Importamos las librerías a utilizar
import cv2
from ultralytics import YOLO
from IPython.display import Image

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n')  # This will automatically download the model weights

# Perform inference
source_img = cv2.imread('foto_img_detect.jpg')
results = model(source_img)

# Visualize the results on the frame
annotated_frame = results[0].plot()

# Guardamos el resultado en JPG
cv2.imwrite('results.jpg', annotated_frame)

# Mostramos el resultado en Colab
Image('results.jpg')

La localización de objeto se realizó de manera satisfactoria, solamente veo que en el caso de la camioneta que esta medio tapada la detectó como si fueran dos autos distintos. Los demás objetos observo que se han detectado bien.

Me resulta interesante que se haya localizado la mochila de forma correcta, eso me sorprendió.

# Ejercicio 8

En lo posible, realizar videos donde una clase de objeto difiera en número con otros objetos (sino, utilizar videos bajados de internet). Por ejemplo, al aire libre, un perro que se mueva entre personas en un parque. Una persona en bicicleta entre muchos autos en una calle de la ciudad. Luego, utilizar los distintos algoritmos presentados en teoría para detectar solo la clase minoritaria con su respectiva etiqueta. Indicando en cada caso la herramienta utilizada y comparando los niveles de confianza de detección logrados.


En este caso voy a utilizar un video de youtube de un bicimensajero pedaleando por las calles de NY.

In [ ]:
%%capture
!pip install pytube
!apt update && apt install -y handbrake

In [ ]:
from pytube import YouTube

# URL del video de YouTube
url = 'https://www.youtube.com/watch?v=Q3sJOToxseU'

# Crear un objeto YouTube
yt = YouTube(url)

# Seleccionar el stream con resolución de 720p
video = yt.streams.filter(progressive=True, file_extension='mp4', res="720p").first()

# Nombre del archivo de salida
output_filename = 'video_ejer_8.mp4'

# Descargar el video con el nombre especificado
video.download(filename=output_filename)

print("Descarga completada.")

In [ ]:
%%capture
#Esto convierte el video a un formato más liviano de trabajo
# con -ss 126 indico el recorte que deseo del video (desde el min 2:06)
# ya que en ese momento del video se visualiza solo un ciclista por la calle (la clase única y minoritaria en este caso)

!ffmpeg -y -i video_ejer_8.mp4 -ss 126 -vf "scale=600:-1" -an -t 15 video_ejer_8_600.mp4


Descargo el video con el formato y resolución modificados

In [ ]:
# from google.colab import files

# # Descargar el archivo de video
# files.download('/content/video_ejer_8_600.mp4')

In [ ]:
!pip install -q mediapy
import mediapy as media

Visualizo el video en formato ya modificado

In [ ]:
url = 'video_ejer_8_600.mp4'
video = media.read_video(url)
media.show_video(video)

Inferencia sobre video aplicando modelo YOLO

In [ ]:
%pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
! [ ! -f video_ejer_8_600.mp4 ] &&  wget https://raw.githubusercontent.com/euglpz/computer_vision/main/unidad_1/video_ejer_8_600.mp4
!yolo predict model=yolov8n.pt source='video_ejer_8_600.mp4' save=True project=video_ejer_8_yolov8

In [ ]:
# Convertimos a MP4 para comprimir el video y optimizar el uso de memoria para la reproducción con mediapy
%%capture
!ffmpeg -i 'video_ejer_8_yolov8/predict/video_ejer_8_600.avi' -vcodec libx264 'video_ejer_8_yolov8/predict/video_ejer_8_600.mp4' -y

Visualización del modelo YOLO funcionando

In [ ]:
# !pip install -q mediapy
# import mediapy as media
url = 'video_ejer_8_yolov8/predict/video_ejer_8_600.mp4'
video = media.read_video(url)
media.show_video(video)

Se pueden notar las localizaciones y segmentaciones de los objetos que se detectan a partir del modelo YOLO, como así también las etiquetas de esos objetos detectados en todas sus instancias.

En este caso me interesa detectar solamente el objeto de clase **'bicycle'**.

Para ello utilizo nuevamente el modelo YOLO y también las funciones desarrolladas en teoría para la detección de objeto y procesamiento de video.

A la hora de detectar el objeto, se le pasa como parámetro a la función de detección solamente la clase 'bicycle', para que de esta forma solo se realice la detección y segmentación de esta clase en el video.

In [ ]:
import mediapy as media  # Importar la biblioteca mediapy para manejo de medios
import cv2  # Importar OpenCV para el procesamiento de imágenes y videos
import numpy as np

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n')  # This will automatically download the model weights

# Función para detectar objetos en una imagen:
def detect_objects(new_image, prev_image, **kwargs):
    # Convertir la imagen a float32
    results = model(new_image)

    # Clases de interés para graficar
    classes = kwargs.get('classes', ['car','person','bicycle', 'bus'])

    # Iteramos sobre los boung boxes obtenidos
    for box in results[0].boxes:
        # Extrayendo los datos del tensor
        x1, y1, x2, y2, confidence, cls = box.data[0]

        # Obteniendo el nombre de la clase
        class_name = model.names[int(cls)]

        # Parámetros opcionales del bounding box
        color = kwargs.get('color', (0, 255, 0))
        thickness = kwargs.get('thickness', 2)

        print(f"Clase: {class_name}, Box: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f}), Confianza: {confidence:.2f}")

        if class_name in classes:
            # Dibujar el rectángulo
            cv2.rectangle(new_image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)

            # Agregar el texto de la confianza
            confidence_text = f"{class_name}: {confidence:.2f}"
            cv2.putText(new_image, confidence_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return new_image

# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas
            prev_image = None  # Inicializar la imagen previa

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # Actualizar la imagen previa
                prev_image = new_image.copy()

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

In [ ]:
# Nombres de los archivos de video de entrada y salida
filename_in = 'video_ejer_8_600.mp4'
filename_out = 'video_ejer_8_600_bboxes.mp4'
parameters = dict(classes=['bicycle'])

# Llamar a la función para procesar el video
video_processor(filename_in, filename_out, detect_objects, 20, **parameters)

# Mostrar el video resultante
media.show_video(media.read_video(filename_out), fps=30)

Se puede visualizar que de esta forma se pudo localizar correctamente a la clase minoritaria (bicicleta) en el video, dejando de lado todos los demás objetos.

In [ ]:
!wget -O 'MobileNetSSD_deploy.prototxt' https://raw.githubusercontent.com/TheNsBhasin/DNN_Object_Detection/master/MobileNetSSD_deploy.prototxt.txt
!wget -O 'MobileNetSSD_deploy.caffemodel' https://github.com/TheNsBhasin/DNN_Object_Detection/blob/master/MobileNetSSD_deploy.caffemodel?raw=true

In [ ]:

# Make sure the video file is in the same directory as your code
filename = 'video_ejer_8_600.mp4'
file_size = (1920,1080) # Assumes 1920x1080 mp4

# We want to save the output to a video file
output_filename = 'mobile_net.mp4'
output_frames_per_second = 20.0

RESIZED_DIMENSIONS = (300, 300) # Dimensions that SSD was trained on.
IMG_NORM_RATIO = 0.007843 # In grayscale a pixel can range between 0 and 255

# Load the pre-trained neural network
neural_network = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt',
        'MobileNetSSD_deploy.caffemodel')

# List of categories and classes
categories = { 0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird',
               4: 'boat', 5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat',
               9: 'chair', 10: 'cow', 11: 'diningtable', 12: 'dog',
              13: 'horse', 14: 'motorbike', 15: 'person',
              16: 'pottedplant', 17: 'sheep', 18: 'sofa',
              19: 'train', 20: 'tvmonitor'}

classes =  ["background", "aeroplane", "bicycle", "bird", "boat", "bottle",
            "bus", "car", "cat", "chair", "cow",
           "diningtable",  "dog", "horse", "motorbike", "person",
           "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# Create the bounding boxes
bbox_colors = np.random.uniform(255, 0, size=(len(categories), 3))

def main():

  # Load a video
  cap = cv2.VideoCapture(filename)

  # Create a VideoWriter object so we can save the video output
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  result = cv2.VideoWriter(output_filename,
                           fourcc,
                           output_frames_per_second,
                           file_size)

  # Process the video
  while cap.isOpened():

    # Capture one frame at a time
    success, frame = cap.read()

    # Do we have a video frame? If true, proceed.
    if success:

      # Capture the frame's height and width
      (h, w) = frame.shape[:2]

      # Create a blob. A blob is a group of connected pixels in a binary
      # frame that share some common property (e.g. grayscale value)
      # Preprocess the frame to prepare it for deep learning classification
      frame_blob = cv2.dnn.blobFromImage(cv2.resize(frame, RESIZED_DIMENSIONS),
                     IMG_NORM_RATIO, RESIZED_DIMENSIONS, 127.5)

      # Set the input for the neural network
      neural_network.setInput(frame_blob)

      # Predict the objects in the image
      neural_network_output = neural_network.forward()

      # Put the bounding boxes around the detected objects
      for i in np.arange(0, neural_network_output.shape[2]):

        confidence = neural_network_output[0, 0, i, 2]

        # Confidence must be at least 30%
        if confidence > 0.20:

          idx = int(neural_network_output[0, 0, i, 1])

          bounding_box = neural_network_output[0, 0, i, 3:7] * np.array(
            [w, h, w, h])

          (startX, startY, endX, endY) = bounding_box.astype("int")

          label = "{}: {:.2f}%".format(classes[idx], confidence * 100)

          cv2.rectangle(frame, (startX, startY), (
            endX, endY), bbox_colors[idx], 2)

          y = startY - 15 if startY - 15 > 15 else startY + 15

          cv2.putText(frame, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX,
            0.5, bbox_colors[idx], 2)

      # We now need to resize the frame so its dimensions
      # are equivalent to the dimensions of the original frame
      frame = cv2.resize(frame, file_size, interpolation=cv2.INTER_NEAREST)

            # Write the frame to the output video file
      result.write(frame)

    # No more video frames left
    else:
      break

  # Stop when the video is finished
  cap.release()

  # Release the video recording
  result.release()

main()

Probando el método MobileNet SSD

In [ ]:
import cv2
import numpy as np

modelo = 'MobileNetSSD_deploy.caffemodel'
configuracion = 'MobileNetSSD_deploy.prototxt'
clases = ["background", "aeroplane", "bicycle", "bird", "boat",
          "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
          "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
          "sofa", "train", "tvmonitor"]

# Cargar la red neuronal
net = cv2.dnn.readNetFromCaffe(configuracion, modelo)

# Capturar el video desde una fuente (puedes cambiar el nombre del archivo o usar 0 para la cámara web)
video = cv2.VideoCapture('video_ejer_8_600.mp4')

# Obtener información del video
fps = int(video.get(cv2.CAP_PROP_FPS))
ancho = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
alto = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Definir el codec y crear el objeto VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_salida = cv2.VideoWriter('mobile_net.mp4', fourcc, fps, (ancho, alto))

bbox_colors = np.random.uniform(255, 0, size=(len(categories), 3))

# Iterar sobre cada fotograma del video
while True:
    # Leer el siguiente fotograma del video
    ret, frame = video.read()
    if not ret:
        break

    # Preparar el fotograma para la detección
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    # Pasar el blob a través de la red y obtener las detecciones
    net.setInput(blob)
    detecciones = net.forward()

    # Iterar sobre las detecciones
    for i in np.arange(0, detecciones.shape[2]):
        confianza = detecciones[0, 0, i, 2]

        # Filtrar detecciones débiles
        if confianza > 0.2:
            idx = int(detecciones[0, 0, i, 1])
            box = detecciones[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Dibujar la predicción en el fotograma
            etiqueta = "{}: {:.2f}%".format(clases[idx], confianza * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY), bbox_colors[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, etiqueta, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_colors[idx], 2)

    # Escribir el fotograma con detecciones en el video de salida
    video_salida.write(frame)

# Liberar los recursos y cerrar las ventanas
video.release()
video_salida.release()
cv2.destroyAllWindows()

Visualizamos el video generado

In [ ]:
media.show_video(media.read_video("/content/mobile_net.mp4"), fps=30)

Probando MobileNetSSD solo para la clase 'bicycle'

In [ ]:
import cv2
import numpy as np

modelo = 'MobileNetSSD_deploy.caffemodel'
configuracion = 'MobileNetSSD_deploy.prototxt'
clases = ["background", "aeroplane", "bicycle", "bird", "boat",
          "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
          "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
          "sofa", "train", "tvmonitor"]
clase_interes = "bicycle"

# Cargar la red neuronal
net = cv2.dnn.readNetFromCaffe(configuracion, modelo)

# Capturar el video desde una fuente (puedes cambiar el nombre del archivo o usar 0 para la cámara web)
video = cv2.VideoCapture('video_ejer_8_600.mp4')

# Obtener información del video
fps = int(video.get(cv2.CAP_PROP_FPS))
ancho = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
alto = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Definir el codec y crear el objeto VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_salida = cv2.VideoWriter('mobile_net_bicycle.mp4', fourcc, fps, (ancho, alto))

bbox_colors = np.random.uniform(255, 0, size=(len(categories), 3))

# Iterar sobre cada fotograma del video
while True:
    # Leer el siguiente fotograma del video
    ret, frame = video.read()
    if not ret:
        break

    # Preparar el fotograma para la detección
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    # Pasar el blob a través de la red y obtener las detecciones
    net.setInput(blob)
    detecciones = net.forward()

    # Iterar sobre las detecciones
    for i in np.arange(0, detecciones.shape[2]):
        confianza = detecciones[0, 0, i, 2]

        # Filtrar detecciones débiles
        if confianza > 0.2:
            idx = int(detecciones[0, 0, i, 1])
            if clase_interes in clases[idx]:
              box = detecciones[0, 0, i, 3:7] * np.array([w, h, w, h])
              (startX, startY, endX, endY) = box.astype("int")

              # Dibujar la predicción en el fotograma
              etiqueta = "{}: {:.2f}%".format(clases[idx], confianza * 100)
              cv2.rectangle(frame, (startX, startY), (endX, endY), bbox_colors[idx], 2)
              y = startY - 15 if startY - 15 > 15 else startY + 15
              cv2.putText(frame, etiqueta, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bbox_colors[idx], 2)

    # Escribir el fotograma con detecciones en el video de salida
    video_salida.write(frame)

# Liberar los recursos y cerrar las ventanas
video.release()
video_salida.release()
cv2.destroyAllWindows()


In [ ]:
media.show_video(media.read_video("/content/mobile_net_bicycle.mp4"), fps=30)

Utilizando el modelo de CNN MobileNetSSD vemos que solamente al comienzo del video puede localizar a la bicicleta, esto puede ocurrir porque quizás para el entrenamiento del modelo no se le pasaron muchas imágenes de bicicletas desde distintos ángulos, por ahí este modelo no es tan robusto para esa clase de objeto, como si lo es el modelo YOLO.

# Ejercicio 9

Tomar fotografías donde coexistan varios objetos en posiciones solapadas y no, en contextos de diferente complejidad. Luego, aplicar los algoritmos de segmentación propuestos y verificar los resultados de cada uno. Comentar qué diferencias observa.

Para este ejercicio voy a utilizar imágenes tomadas en la calle en distintas situaciones para ver como se comportan los distintos modelos.

##Instance Segmentation con Mask R-CNN (OpenCV)

In [ ]:
%%capture
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!tar zxvf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!wget https://raw.githubusercontent.com/spmallick/learnopencv/master/Mask-RCNN/mscoco_labels.names -O mscoco_labels.names
!wget https://raw.githubusercontent.com/spmallick/learnopencv/master/Mask-RCNN/colors.txt -O colors.txt
!wget https://raw.githubusercontent.com/spmallick/learnopencv/master/Mask-RCNN/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt -O mask_rcnn_inception_v2_coco_2018_01_28.pbtxt

In [ ]:
import numpy as np
import cv2 as cv
import time
import random
from matplotlib import pyplot as plt

# Umbral de confianza para las detecciones
confThreshold = 0.5
# Umbral para las máscaras de segmentación
maskThreshold = 0.3

# Cargar los nombres de las clases de MSCOCO
classesFile = "mscoco_labels.names"
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Cargar los colores para visualizar las detecciones
colorsFile = "colors.txt"
colors = []
with open(colorsFile, 'rt') as f:
    colorsStr = f.read().rstrip('\n').split('\n')
    for i in range(len(colorsStr)):
        rgb = colorsStr[i].split(' ')
        color = np.array([float(rgb[0]), float(rgb[1]), float(rgb[2])])
        colors.append(color)

# Archivos de configuración y pesos del modelo Mask R-CNN
textGraph = "./mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"
modelWeights = "./mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb"

# Cargar la red
net = cv.dnn.readNetFromTensorflow(modelWeights, textGraph)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# Cargar la imagen y obtener sus dimensiones
image_1 = cv.imread('mza1.jpg')
(H, W) = image_1.shape[:2]

# Crear un blob de la imagen y realizar una pasada hacia adelante con Mask R-CNN
blob = cv.dnn.blobFromImage(image_1, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

# Información de tiempo y formas de las detecciones y máscaras
print("[INFO] Mask R-CNN tomó {:.6f} segundos en procesar la imagen".format(end - start))
print("[INFO] Forma de boxes: {}".format(boxes.shape))
print("[INFO] Forma de masks: {}".format(masks.shape))

In [ ]:
classes
image_1.shape[:2]

In [ ]:
# Creamos una copia de la imagen original, para dibujar las máscaras
output_image = image_1.copy()

for i in range(0, boxes.shape[2]):
    confidence = boxes[0, 0, i, 2]
    if confidence > confThreshold:
        classId = int(boxes[0, 0, i, 1])
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")

        # Redimensiona la máscara para que se ajuste a la caja delimitadora y la convierte en uint8
        mask = masks[i, classId]
        mask = cv.resize(mask, (endX - startX, endY - startY))
        mask = (mask > maskThreshold).astype("uint8")

        # Extrae la región de interés (ROI) usando las coordenadas de la caja delimitadora
        roi = output_image[startY:endY, startX:endX]

        # Crea una imagen de color sólido para la máscara
        color = random.choice(colors)
        color_mask = np.zeros_like(roi)
        color_mask[mask == 1] = color

        # Mezcla la imagen de color con la ROI utilizando la máscara
        roi = cv.addWeighted(roi, 1, color_mask, 0.4, 0)

        # Vuelve a colocar la ROI en la imagen original
        output_image[startY:endY, startX:endX] = roi

        # Dibuja la caja delimitadora
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)

        # Consigue el nombre de la clase y el color correspondiente
        classId = int(boxes[0, 0, i, 1])
        className = classes[classId]

        # Dibuja el rectángulo de la caja delimitadora y la etiqueta de la clase
        label = f"{className}: {confidence:.2f}"
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)
        cv.putText(output_image, label, (startX, startY - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


# Configurar los subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Mostrar la imagen original
axes[0].imshow(cv.cvtColor(image_1, cv.COLOR_BGR2RGB))
axes[0].set_title('Imagen Original')
axes[0].axis('off')  # Ocultar los ejes

# Mostrar la imagen segmentada
axes[1].imshow(cv.cvtColor(output_image, cv.COLOR_BGR2RGB))
axes[1].set_title('Imagen Segmentada')
axes[1].axis('off')  # Ocultar los ejes

# Mostrar ambas imágenes
plt.show()

In [ ]:
# Cargar la imagen y obtener sus dimensiones
image_2 = cv.imread('mza2.jpg')
(H2, W2) = image_2.shape[:2]

# Crear un blob de la imagen y realizar una pasada hacia adelante con Mask R-CNN
blob = cv.dnn.blobFromImage(image_2, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

# Información de tiempo y formas de las detecciones y máscaras
print("[INFO] Mask R-CNN tomó {:.6f} segundos en procesar la imagen".format(end - start))
print("[INFO] Forma de boxes: {}".format(boxes.shape))
print("[INFO] Forma de masks: {}".format(masks.shape))

In [ ]:
# Creamos una copia de la imagen original, para dibujar las máscaras
output_image = image_2.copy()

for i in range(0, boxes.shape[2]):
    confidence = boxes[0, 0, i, 2]
    if confidence > confThreshold:
        classId = int(boxes[0, 0, i, 1])
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")

        # Redimensiona la máscara para que se ajuste a la caja delimitadora y la convierte en uint8
        mask = masks[i, classId]
        mask = cv.resize(mask, (endX - startX, endY - startY))
        mask = (mask > maskThreshold).astype("uint8")

        # Extrae la región de interés (ROI) usando las coordenadas de la caja delimitadora
        roi = output_image[startY:endY, startX:endX]

        # Crea una imagen de color sólido para la máscara
        color = random.choice(colors)
        color_mask = np.zeros_like(roi)
        color_mask[mask == 1] = color

        # Mezcla la imagen de color con la ROI utilizando la máscara
        roi = cv.addWeighted(roi, 1, color_mask, 0.4, 0)

        # Vuelve a colocar la ROI en la imagen original
        output_image[startY:endY, startX:endX] = roi

        # Dibuja la caja delimitadora
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)

        # Consigue el nombre de la clase y el color correspondiente
        classId = int(boxes[0, 0, i, 1])
        className = classes[classId]

        # Dibuja el rectángulo de la caja delimitadora y la etiqueta de la clase
        label = f"{className}: {confidence:.2f}"
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)
        cv.putText(output_image, label, (startX, startY - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


# Configurar los subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Mostrar la imagen original
axes[0].imshow(cv.cvtColor(image_2, cv.COLOR_BGR2RGB))
axes[0].set_title('Imagen Original')
axes[0].axis('off')  # Ocultar los ejes

# Mostrar la imagen segmentada
axes[1].imshow(cv.cvtColor(output_image, cv.COLOR_BGR2RGB))
axes[1].set_title('Imagen Segmentada')
axes[1].axis('off')  # Ocultar los ejes

# Mostrar ambas imágenes
plt.show()

In [ ]:
image_3 = cv.imread('sj1.jpg')
(H2, W2) = image_3.shape[:2]

# Crear un blob de la imagen y realizar una pasada hacia adelante con Mask R-CNN
blob = cv.dnn.blobFromImage(image_2, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

# Información de tiempo y formas de las detecciones y máscaras
print("[INFO] Mask R-CNN tomó {:.6f} segundos en procesar la imagen".format(end - start))
print("[INFO] Forma de boxes: {}".format(boxes.shape))
print("[INFO] Forma de masks: {}".format(masks.shape))

In [ ]:
output_image = image_3.copy()

for i in range(0, boxes.shape[2]):
    confidence = boxes[0, 0, i, 2]
    if confidence > confThreshold:
        classId = int(boxes[0, 0, i, 1])
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")

        # Redimensiona la máscara para que se ajuste a la caja delimitadora y la convierte en uint8
        mask = masks[i, classId]
        mask = cv.resize(mask, (endX - startX, endY - startY))
        mask = (mask > maskThreshold).astype("uint8")

        # Extrae la región de interés (ROI) usando las coordenadas de la caja delimitadora
        roi = output_image[startY:endY, startX:endX]

        # Crea una imagen de color sólido para la máscara
        color = random.choice(colors)
        color_mask = np.zeros_like(roi)
        color_mask[mask == 1] = color

        # Mezcla la imagen de color con la ROI utilizando la máscara
        roi = cv.addWeighted(roi, 1, color_mask, 0.4, 0)

        # Vuelve a colocar la ROI en la imagen original
        output_image[startY:endY, startX:endX] = roi

        # Dibuja la caja delimitadora
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)

        # Consigue el nombre de la clase y el color correspondiente
        classId = int(boxes[0, 0, i, 1])
        className = classes[classId]

        # Dibuja el rectángulo de la caja delimitadora y la etiqueta de la clase
        label = f"{className}: {confidence:.2f}"
        cv.rectangle(output_image, (startX, startY), (endX, endY), color, 2)
        cv.putText(output_image, label, (startX, startY - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


# Configurar los subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Mostrar la imagen original
axes[0].imshow(cv.cvtColor(image_3, cv.COLOR_BGR2RGB))
axes[0].set_title('Imagen Original')
axes[0].axis('off')  # Ocultar los ejes

# Mostrar la imagen segmentada
axes[1].imshow(cv.cvtColor(output_image, cv.COLOR_BGR2RGB))
axes[1].set_title('Imagen Segmentada')
axes[1].axis('off')  # Ocultar los ejes

# Mostrar ambas imágenes
plt.show()

## Instance Segmentation con modelo PointRend Resnet50 (PixelLib)

In [ ]:
%%capture
!pip install pixellib torchvision
# Instance segmentation model
!wget "https://github.com/ayoolaolafenwa/PixelLib/releases/download/0.2.0/pointrend_resnet50.pkl" -O pointrend_resnet50.pkl

In [ ]:
# Instalación de la biblioteca PixelLib
# !pip install pixellib

# Importación de las bibliotecas necesarias
import pixellib
from pixellib.torchbackend.instance import instanceSegmentation
from IPython.display import Image

# Creación de una instancia del modelo de segmentación
ins = instanceSegmentation()

# Carga del modelo preentrenado
ins.load_model("pointrend_resnet50.pkl")

# Segmentación de la imagen y guardado de la imagen segmentada con las cajas delimitadoras
ins.segmentImage("mza1.jpg", show_bboxes=True, output_image_name="mza1_segmented.jpg")

# Muestra la imagen original
display(Image(filename="mza1.jpg"))

# Muestra la imagen segmentada
display(Image(filename="mza1_segmented.jpg"))


In [ ]:
# Segmentación de la imagen y guardado de la imagen segmentada con las cajas delimitadoras
ins.segmentImage("mza2.jpg", show_bboxes=True, output_image_name="mza2_segmented.jpg")

# Muestra la imagen original
display(Image(filename="mza2.jpg"))

# Muestra la imagen segmentada
display(Image(filename="mza2_segmented.jpg"))


In [ ]:
ins.segmentImage("sj1.jpg", show_bboxes=True, output_image_name="sj1_segmented.jpg")

# Muestra la imagen original
display(Image(filename="sj1.jpg"))

# Muestra la imagen segmentada
display(Image(filename="sj1_segmented.jpg"))

## Semantic Segmentation con modelo DeepLabV3 (Torchvision)

In [ ]:
%%capture
!pip install torchvision
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import torch

# Carga y procesa la imagen
input_image = Image.open("mza1.jpg").convert("RGB").convert("RGB")

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # crea un mini-batch como espera el modelo

# Mueve el input y el modelo a GPU para mayor velocidad si está disponible
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

# Crea una paleta de colores, seleccionando un color para cada clase
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# Genera la imagen de segmentación semántica
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
r.putpalette(colors)

# Muestra la imagen original y la segmentada lado a lado
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # 1 fila, 2 columnas, primer subplot
plt.imshow(input_image)
plt.title('Imagen Original')
plt.axis('off')  # Oculta los ejes

plt.subplot(1, 2, 2)  # 1 fila, 2 columnas, segundo subplot
plt.imshow(r)
plt.title('Imagen Segmentada')
plt.axis('off')  # Oculta los ejes

plt.show()

In [ ]:
# Carga y procesa la imagen
input_image = Image.open("mza2.jpg").convert("RGB").convert("RGB")

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # crea un mini-batch como espera el modelo

# Mueve el input y el modelo a GPU para mayor velocidad si está disponible
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

# Crea una paleta de colores, seleccionando un color para cada clase
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# Genera la imagen de segmentación semántica
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
r.putpalette(colors)

# Muestra la imagen original y la segmentada lado a lado
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # 1 fila, 2 columnas, primer subplot
plt.imshow(input_image)
plt.title('Imagen Original')
plt.axis('off')  # Oculta los ejes

plt.subplot(1, 2, 2)  # 1 fila, 2 columnas, segundo subplot
plt.imshow(r)
plt.title('Imagen Segmentada')
plt.axis('off')  # Oculta los ejes

plt.show()

In [ ]:
# Carga y procesa la imagen
input_image = Image.open("sj1.jpg").convert("RGB").convert("RGB")

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # crea un mini-batch como espera el modelo

# Mueve el input y el modelo a GPU para mayor velocidad si está disponible
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

# Crea una paleta de colores, seleccionando un color para cada clase
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# Genera la imagen de segmentación semántica
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
r.putpalette(colors)

# Muestra la imagen original y la segmentada lado a lado
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # 1 fila, 2 columnas, primer subplot
plt.imshow(input_image)
plt.title('Imagen Original')
plt.axis('off')  # Oculta los ejes

plt.subplot(1, 2, 2)  # 1 fila, 2 columnas, segundo subplot
plt.imshow(r)
plt.title('Imagen Segmentada')
plt.axis('off')  # Oculta los ejes

plt.show()

##Instance Segmentation con PSPNet (MMSegmentation)

In [ ]:
%%capture
%%bash
pip install -U openmim
mim install mmengine
mim install "mmcv>=2.0.0"
pip install "mmsegmentation>=1.0.0" ftfy
mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest .


In [ ]:
!git clone -b main https://github.com/open-mmlab/mmsegmentation.git

In [ ]:
%%capture
# Instalación de librería MMSegmentation
import sys
!cd /content/mmsegmentation && {sys.executable} -m pip install -v -e .

In [ ]:
import os
os.chdir('/content/computer_vision/unidad_1/mmsegmentation')

In [ ]:
pip install "mmcv>=2.0.0rc4"

In [ ]:

from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
import matplotlib.pyplot as plt
import numpy as np

# Configuración e inicialización del modelo
config_file = '../pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
checkpoint_file = '../pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'
model = init_model(config_file, checkpoint_file, device='cuda:0')

# Ruta de la imagen a segmentar
img_path = '/content/mza1.jpg'

# Inferencia del modelo
result = inference_model(model, img_path)

# display the segmentation result
segmented_image = show_result_pyplot(model, img_path, result)

# Visualización con matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(segmented_image)
plt.axis('off')  # Oculta los ejes
plt.show()

## Instance Segmentation con YoloV8 (Ultralytics)

In [ ]:
%%capture
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
from IPython.display import display

# Cargar el modelo YOLOv8 con capacidad de segmentación
model = YOLO("yolov8m-seg.pt")

# Realizar predicciones en la imagen
results = model.predict('mza1.jpg')

# Seleccionar el primer resultado (en caso de que haya varios)
result = results[0]

# Abrir la imagen original para dibujar sobre ella
img = Image.open("mza1.jpg")
draw = ImageDraw.Draw(img)

# Para cada detección, dibujar el bounding box y mostrar la clase
for box in results[0].boxes:
    # Extracción de coordenadas del bounding box y clase
    x1, y1, x2, y2, conf, cls = box.data[0]
    label = result.names[int(cls)]  # Asumiendo que result.names contiene los nombres de las clases

    # Dibujar el bounding box
    draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=2)

    # Mostrar la clase (y opcionalmente la confianza) en el bounding box
    text = f"{label} {conf:.2f}"
    draw.text((x1, y1), text, fill=(0, 255, 0))

# Acceder a las máscaras del resultado
masks = result.masks

# Iterar a través de todas las máscaras encontradas
for mask in masks:
    # Convertir la máscara a un array numpy y obtener el polígono
    mask_data = mask.data[0].numpy()
    polygon = mask.xy[0]

    # Dibujar el polígono correspondiente a cada máscara en la imagen original
    polygon_list = [(p[0], p[1]) for p in polygon]
    draw.polygon(polygon_list, outline=(255, 0, 0), width=3)

# Mostrar la imagen con los polígonos dibujados
display(img)

In [ ]:
# Cargar el modelo YOLOv8 con capacidad de segmentación
model = YOLO("yolov8m-seg.pt")

# Realizar predicciones en la imagen
results = model.predict('mza2.jpg')

# Seleccionar el primer resultado (en caso de que haya varios)
result = results[0]

# Abrir la imagen original para dibujar sobre ella
img = Image.open("mza2.jpg")
draw = ImageDraw.Draw(img)

# Para cada detección, dibujar el bounding box y mostrar la clase
for box in results[0].boxes:
    # Extracción de coordenadas del bounding box y clase
    x1, y1, x2, y2, conf, cls = box.data[0]
    label = result.names[int(cls)]  # Asumiendo que result.names contiene los nombres de las clases

    # Dibujar el bounding box
    draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=2)

    # Mostrar la clase (y opcionalmente la confianza) en el bounding box
    text = f"{label} {conf:.2f}"
    draw.text((x1, y1), text, fill=(0, 255, 0))

# Acceder a las máscaras del resultado
masks = result.masks

# Iterar a través de todas las máscaras encontradas
for mask in masks:
    # Convertir la máscara a un array numpy y obtener el polígono
    mask_data = mask.data[0].numpy()
    polygon = mask.xy[0]

    # Dibujar el polígono correspondiente a cada máscara en la imagen original
    polygon_list = [(p[0], p[1]) for p in polygon]
    draw.polygon(polygon_list, outline=(255, 0, 0), width=3)

# Mostrar la imagen con los polígonos dibujados
display(img)

In [ ]:
# Cargar el modelo YOLOv8 con capacidad de segmentación
model = YOLO("yolov8m-seg.pt")

# Realizar predicciones en la imagen
results = model.predict('sj1.jpg')

# Seleccionar el primer resultado (en caso de que haya varios)
result = results[0]

# Abrir la imagen original para dibujar sobre ella
img = Image.open("sj1.jpg")
draw = ImageDraw.Draw(img)

# Para cada detección, dibujar el bounding box y mostrar la clase
for box in results[0].boxes:
    # Extracción de coordenadas del bounding box y clase
    x1, y1, x2, y2, conf, cls = box.data[0]
    label = result.names[int(cls)]  # Asumiendo que result.names contiene los nombres de las clases

    # Dibujar el bounding box
    draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=2)

    # Mostrar la clase (y opcionalmente la confianza) en el bounding box
    text = f"{label} {conf:.2f}"
    draw.text((x1, y1), text, fill=(0, 255, 0))

# Acceder a las máscaras del resultado
masks = result.masks

# Iterar a través de todas las máscaras encontradas
for mask in masks:
    # Convertir la máscara a un array numpy y obtener el polígono
    mask_data = mask.data[0].numpy()
    polygon = mask.xy[0]

    # Dibujar el polígono correspondiente a cada máscara en la imagen original
    polygon_list = [(p[0], p[1]) for p in polygon]
    draw.polygon(polygon_list, outline=(255, 0, 0), width=3)

# Mostrar la imagen con los polígonos dibujados
display(img)

## Panoptic segmentation con modelo DETR ResNet50 (Transformers)

In [ ]:
%%capture
!pip install timm

In [ ]:
import io
import requests
from PIL import Image, ImageDraw, ImageFont
import torch
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from transformers import DetrFeatureExtractor, DetrForSegmentation
from transformers.models.detr.feature_extraction_detr import rgb_to_id

# Cargar la imagen
img_path = "mza1.jpg"
image = Image.open(img_path)

# Cargar el modelo y el extractor de características
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

# Preparar la imagen para el modelo
inputs = feature_extractor(images=image, return_tensors="pt")

# Paso forward
outputs = model(**inputs)

# Postprocesamiento
processed_sizes = torch.as_tensor(inputs["pixel_values"].shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes, threshold=0.85)[0]

# Convertir la segmentación a numpy
panoptic_seg = Image.open(io.BytesIO(result["png_string"]))
panoptic_seg = np.array(panoptic_seg, dtype=np.uint8)
panoptic_seg_id = rgb_to_id(panoptic_seg)

# Preparar la paleta de colores
palette = itertools.cycle(sns.color_palette())

# Crear la imagen segmentada
segmented_image = Image.fromarray(np.zeros_like(panoptic_seg, dtype=np.uint8))
draw = ImageDraw.Draw(segmented_image)

# Añadimos un mapeo manual de los IDs de las categorías a los nombres, basado en las clases comunes de COCO
COCO_LABELS = {
    1: 'persona', 2: 'bicicleta', 3: 'coche', 4: 'motocicleta', 5: 'avión',
    6: 'autobús', 7: 'tren', 8: 'camión', 9: 'barco', 10: 'semáforo',
    11: 'hidrante', 13: 'señal de stop', 14: 'parquímetro', 15: 'banco', 16: 'pájaro',
    17: 'gato', 18: 'perro', 19: 'caballo', 20: 'oveja', 21: 'vaca',
    22: 'elefante', 23: 'oso', 24: 'cebra', 25: 'jirafa', 27: 'mochila',
    28: 'paraguas', 31: 'bolso de mano', 32: 'corbata', 33: 'maleta', 34: 'frisbee',
    36: 'tabla de snowboard', 37: 'pelota deportiva', 38: 'cometa', 39: 'bate de béisbol',
    40: 'guante de béisbol', 41: 'patineta', 42: 'tabla de surf', 43: 'raqueta de tenis',
    44: 'botella', 46: 'plato de vino', 47: 'taza', 48: 'tenedor', 49: 'cuchillo',
    50: 'cuchara', 51: 'tazón', 52: 'banana', 53: 'manzana', 54: 'sándwich',
    55: 'naranja', 56: 'brócoli', 57: 'zanahoria', 58: 'perrito caliente', 59: 'pizza',
    60: 'donut', 61: 'pastel', 62: 'silla', 63: 'sofá', 64: 'maceta', 65: 'cama',
    67: 'mesa de comedor', 70: 'inodoro', 72: 'TV', 73: 'computadora portátil', 74: 'ratón',
    75: 'control remoto', 76: 'teclado', 77: 'teléfono celular', 78: 'microondas',
    79: 'horno', 80: 'tostadora', 81: 'fregadero', 82: 'refrigerador', 84: 'libro',
    85: 'reloj', 86: 'florero', 87: 'tijeras', 88: 'oso de peluche', 89: 'secador de pelo',
    90: 'cepillo de dientes',
}

# Ajusta el bucle para dibujar segmentos y etiquetas correctamente
for segment_info in result["segments_info"]:
    class_id = segment_info["category_id"]
    class_name = COCO_LABELS.get(class_id, 'Desconocido')  # 'Desconocido' si el ID no está en el diccionario
    id = segment_info["id"]

    # Generar la máscara para este segmento específico
    mask = panoptic_seg_id == id
    color = np.array(next(palette)) * 255  # Convertir el color a un array de numpy adecuado

    # Convertir la máscara a una imagen de PIL para usarla como máscara en 'paste'
    mask_image = Image.fromarray((mask * 255).astype(np.uint8))

    # Crear una imagen del color del segmento que tenga las dimensiones correctas
    color_image = Image.new("RGB", segmented_image.size, color=tuple(color.astype(int)))

    # Pegar usando la máscara para aplicar solo este segmento
    segmented_image.paste(color_image, (0,0), mask=mask_image)

    # Dibujar el nombre de la clase en la posición inicial del segmento
    draw = ImageDraw.Draw(segmented_image)
    where = np.where(mask)
    if where[0].size > 0 and where[1].size > 0:
        x, y = np.min(where[1]), np.min(where[0])
        draw.text((x, y), class_name, fill='white')

# Mostrar la imagen original y la segmentada
plt.figure(figsize=(30, 15))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title('Imagen Segmentada con Etiquetas')
plt.axis('off')

plt.show()


In [ ]:
# Cargar la imagen
img_path = "mza2.jpg"
image = Image.open(img_path)

# Cargar el modelo y el extractor de características
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

# Preparar la imagen para el modelo
inputs = feature_extractor(images=image, return_tensors="pt")

# Paso forward
outputs = model(**inputs)

# Postprocesamiento
processed_sizes = torch.as_tensor(inputs["pixel_values"].shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes, threshold=0.85)[0]

# Convertir la segmentación a numpy
panoptic_seg = Image.open(io.BytesIO(result["png_string"]))
panoptic_seg = np.array(panoptic_seg, dtype=np.uint8)
panoptic_seg_id = rgb_to_id(panoptic_seg)

# Preparar la paleta de colores
palette = itertools.cycle(sns.color_palette())

# Crear la imagen segmentada
segmented_image = Image.fromarray(np.zeros_like(panoptic_seg, dtype=np.uint8))
draw = ImageDraw.Draw(segmented_image)


# Ajusta el bucle para dibujar segmentos y etiquetas correctamente
for segment_info in result["segments_info"]:
    class_id = segment_info["category_id"]
    class_name = COCO_LABELS.get(class_id, 'Desconocido')  # 'Desconocido' si el ID no está en el diccionario
    id = segment_info["id"]

    # Generar la máscara para este segmento específico
    mask = panoptic_seg_id == id
    color = np.array(next(palette)) * 255  # Convertir el color a un array de numpy adecuado

    # Convertir la máscara a una imagen de PIL para usarla como máscara en 'paste'
    mask_image = Image.fromarray((mask * 255).astype(np.uint8))

    # Crear una imagen del color del segmento que tenga las dimensiones correctas
    color_image = Image.new("RGB", segmented_image.size, color=tuple(color.astype(int)))

    # Pegar usando la máscara para aplicar solo este segmento
    segmented_image.paste(color_image, (0,0), mask=mask_image)

    # Dibujar el nombre de la clase en la posición inicial del segmento
    draw = ImageDraw.Draw(segmented_image)
    where = np.where(mask)
    if where[0].size > 0 and where[1].size > 0:
        x, y = np.min(where[1]), np.min(where[0])
        draw.text((x, y), class_name, fill='white')

# Mostrar la imagen original y la segmentada
plt.figure(figsize=(30, 15))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title('Imagen Segmentada con Etiquetas')
plt.axis('off')

plt.show()


In [ ]:
# Cargar la imagen
img_path = "sj1.jpg"
image = Image.open(img_path)

# Cargar el modelo y el extractor de características
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

# Preparar la imagen para el modelo
inputs = feature_extractor(images=image, return_tensors="pt")

# Paso forward
outputs = model(**inputs)

# Postprocesamiento
processed_sizes = torch.as_tensor(inputs["pixel_values"].shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes, threshold=0.85)[0]

# Convertir la segmentación a numpy
panoptic_seg = Image.open(io.BytesIO(result["png_string"]))
panoptic_seg = np.array(panoptic_seg, dtype=np.uint8)
panoptic_seg_id = rgb_to_id(panoptic_seg)

# Preparar la paleta de colores
palette = itertools.cycle(sns.color_palette())

# Crear la imagen segmentada
segmented_image = Image.fromarray(np.zeros_like(panoptic_seg, dtype=np.uint8))
draw = ImageDraw.Draw(segmented_image)


# Ajusta el bucle para dibujar segmentos y etiquetas correctamente
for segment_info in result["segments_info"]:
    class_id = segment_info["category_id"]
    class_name = COCO_LABELS.get(class_id, 'Desconocido')  # 'Desconocido' si el ID no está en el diccionario
    id = segment_info["id"]

    # Generar la máscara para este segmento específico
    mask = panoptic_seg_id == id
    color = np.array(next(palette)) * 255  # Convertir el color a un array de numpy adecuado

    # Convertir la máscara a una imagen de PIL para usarla como máscara en 'paste'
    mask_image = Image.fromarray((mask * 255).astype(np.uint8))

    # Crear una imagen del color del segmento que tenga las dimensiones correctas
    color_image = Image.new("RGB", segmented_image.size, color=tuple(color.astype(int)))

    # Pegar usando la máscara para aplicar solo este segmento
    segmented_image.paste(color_image, (0,0), mask=mask_image)

    # Dibujar el nombre de la clase en la posición inicial del segmento
    draw = ImageDraw.Draw(segmented_image)
    where = np.where(mask)
    if where[0].size > 0 and where[1].size > 0:
        x, y = np.min(where[1]), np.min(where[0])
        draw.text((x, y), class_name, fill='white')

# Mostrar la imagen original y la segmentada
plt.figure(figsize=(30, 15))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title('Imagen Segmentada con Etiquetas')
plt.axis('off')

plt.show()


## Conclusiones

Al probar los distintos modelos de segmentación se obtuvieron diferentes resultados.

De acuerdo a los modelos de segmentación por instancia se observan en todos buenos resultados en cuanto a la segmentación de los objetos que figuran en las imágenes pero en cuanto a mi percepción, veo que el modelo PointRed ResNet de PixelLib no solo segmenta a la perfección cada instancia de la imagen sino que tambien pinta cada uno de un color distinto, que sirve mucho para diferenciar cada objeto, y además agrega la etiqueta de cada clase a la salida.

El modelo de YOLO realiza muy buena segmentación pero no pinta de color las instancias y tampoco agrega etiqueta de la clase.

El modelo Mask R-CNN de OpenCV realiza una buena segmentación en la primera imagen pero en la segunda no logra segmentar bien por ejemplo a la moto.

El modelo de segmentación semántica DeepLabV3 de Torchvision logra realizar una segmentación muy precisa de cada objeto pero por ejemplo en la segunda imagen segmenta parte de un árbol que luego no se logra entender bien de que se trata en el resultado final.

No pude hacer funcionar el modelo MMSegmentation por problemas con la instalación, chequearé si puedo solucionar este tema.

Por último en el modelo de segmentación Panoptic DETR ResNet50 de Transformers se logra ver la combinación entre la segmentacion semántica y de instancia, logrando observar una panorama general de la escena con las etiquetas de clase. Un detalle que veo es que por ahí no se logra apreciar bien del todo lo que está pasando cuando hay algun árbol entre medio o porque no llega a realizar una segmentación de los edificios, pintando todo de un mismo color, agregando también parte del cielo.

# Ejercicio 10

Con el mismo video usado en el ejercicio 4, realice un seguimiento de objetos con el código propuesto en la sección de ejemplos prácticos.

In [ ]:
!pip install -q mediapy
import mediapy as media

url = 'video_600.mp4'
video = media.read_video(url)
media.show_video(video)

In [ ]:
!pip install -q supervision ultralytics

# Importar las librerías necesarias
import supervision as sv  # Librería para el seguimiento de objetos
from ultralytics import YOLO  # Librería para la detección de objetos con YOLO
import numpy as np  # Librería para operaciones matemáticas y manejo de arrays

# Definir rutas de los videos de entrada y salida, y el nombre del modelo
SOURCE_VIDEO_PATH = "video_600.mp4"  # Ruta al video de entrada
TARGET_VIDEO_PATH = "video_600_tracking.mp4"  # Ruta donde se guardará el video con seguimiento de objetos
MODEL_NAME = "yolov8x.pt"  # Nombre del modelo preentrenado de YOLO a utilizar

# Inicializar el modelo de detección y el rastreador de objetos
model = YOLO(MODEL_NAME)  # Cargar el modelo de YOLO
tracker = sv.ByteTrack()  # Inicializar ByteTrack para el seguimiento de objetos

# Inicializar los anotadores para las cajas delimitadoras y las etiquetas
bounding_box_annotator = sv.BoundingBoxAnnotator()  # Para dibujar cajas delimitadoras
label_annotator = sv.LabelAnnotator()  # Para dibujar etiquetas (IDs de seguimiento)

# Definir la función de callback que procesa cada frame
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = model(frame)[0]  # Detectar objetos en el frame actual con YOLO
    detections = sv.Detections.from_ultralytics(results)  # Convertir resultados a formato de supervision
    detections = tracker.update_with_detections(detections)  # Actualizar el estado del rastreador con las detecciones

    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]  # Crear etiquetas con los IDs de seguimiento

    # Anotar el frame con bounding boxes y etiquetas
    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame  # Devolver el frame anotado

# Procesar el video: leer, aplicar callback a cada frame y guardar el resultado
sv.process_video(
    source_path=SOURCE_VIDEO_PATH,  # Ruta del video original
    target_path=TARGET_VIDEO_PATH,  # Ruta del video resultante
    callback=callback  # Función de callback para procesar cada frame
)


In [ ]:
url = 'video_600_tracking.mp4'
video = media.read_video(url)
media.show_video(video)

Se puede ver que este modelo segmenta mucho mejor que las funciones utilizadas en el ejercicio 4. Se podrían mejorar aplicando distintos filtros o cambiando los diferentes parámetros de las funciones de OpenCV utilizadas.